In [1]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import Counter
from tqdm import tqdm
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights

In [2]:
class MultiTaskEfficientNet(nn.Module):
    """
    A multi-task learning model using a pre-trained EfficientNetV2-S as a backbone.
    It has four separate classification heads to predict color, type, season, and gender.
    """
    def __init__(self, num_colors, num_types, num_seasons, num_genders):
        super().__init__()
        # Load pre-trained backbone with the latest recommended weights
        self.backbone = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT)

        # Freeze all parameters in the backbone
        for param in self.backbone.parameters():
            param.requires_grad = False
        
        # Get the number of input features for the classifier
        n_features = self.backbone.classifier[1].in_features
        # Replace the classifier with an Identity layer to get the features
        self.backbone.classifier = nn.Identity()

        # Define separate heads for each task, with Dropout for regularization
        self.color_head = nn.Sequential(nn.Dropout(p=0.4), nn.Linear(n_features, num_colors))
        self.type_head = nn.Sequential(nn.Dropout(p=0.4), nn.Linear(n_features, num_types))
        self.season_head = nn.Sequential(nn.Dropout(p=0.4), nn.Linear(n_features, num_seasons))
        self.gender_head = nn.Sequential(nn.Dropout(p=0.4), nn.Linear(n_features, num_genders))

    def forward(self, x):
        # Pass input through the backbone to get shared features
        features = self.backbone(x)
        # Pass features through each head to get task-specific outputs
        return {
            'color': self.color_head(features),
            'product_type': self.type_head(features),
            'season': self.season_head(features),
            'gender': self.gender_head(features)
        }

In [3]:
def get_train_augs():
    """Defines the augmentation pipeline for the training set."""
    return A.Compose([
        A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.33), p=1.0),
        A.HorizontalFlip(p=0.5),
        A.ColorJitter(p=0.7, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        A.CoarseDropout(max_holes=1, max_height=64, max_width=64, p=0.5),


        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

def get_val_augs():
    """Defines the augmentation pipeline for the validation/test set."""
    return A.Compose([
        A.Resize(height=224, width=224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

In [4]:
class FashionDataset(Dataset):
    """Custom PyTorch Dataset for loading fashion product images and labels."""
    def __init__(self, df, img_dir, transform=None):
        self.data = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        # Get the row from the dataframe
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{row['id']}.jpg")
        
        # Open image and convert to RGB
        image = Image.open(img_path).convert('RGB')
        
        # Apply transformations if they exist
        if self.transform:
            image_np = np.array(image)
            augmented = self.transform(image=image_np)
            image = augmented['image']

        # Get the labels
        labels = {
            'color': torch.tensor(row['color_label'], dtype=torch.long),
            'product_type': torch.tensor(row['type_label'], dtype=torch.long),
            'season': torch.tensor(row['season_label'], dtype=torch.long),
            'gender': torch.tensor(row['gender_label'], dtype=torch.long)
        }
        
        return image, labels

In [ ]:
# --- 1. Configuration ---
csv_path = r'D:\CODING\Machine Learning\PROJECTS\fashion-product-classifier\data\processed\cleaned-styles.csv'
img_dir = r'D:\CODING\Machine Learning\PROJECTS\fashion-product-classifier\data\raw\fashion-product-images-dataset\images'
model_save_path = r'D:\CODING\Machine Learning\PROJECTS\fashion-product-classifier\backend\models\checkpoints\model_efficientnet.pth'
encoder_save_path = r'D:\CODING\Machine Learning\PROJECTS\fashion-product-classifier\backend\models\encodings\label_encoders.json'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")
os.makedirs('./backend/models/', exist_ok=True)

# --- 2. Data Loading and Preprocessing ---
df = pd.read_csv(csv_path)

# Create and save label encoders
encoders = {}
label_cols = ['baseColour', 'articleType', 'season', 'gender']
for col in label_cols:
    le = LabelEncoder()
    # Create new column with encoded labels
    df[f'{col.replace("baseColour", "color").replace("articleType", "type")}_label'] = le.fit_transform(df[col])
    # Store encoder classes for decoding later
    encoders[col] = {str(i): c for i, c in enumerate(le.classes_)}

with open(encoder_save_path, 'w') as f:
    json.dump(encoders, f, indent=4)
print(f" Label encoders saved to {encoder_save_path}")

num_classes = {
    'colors': len(encoders['baseColour']),
    'types': len(encoders['articleType']),
    'seasons': len(encoders['season']),
    'genders': len(encoders['gender'])
}

# --- 3. Train/Validation Split ---
# Stratify by 'articleType' to ensure balanced classes in both splits
# Remove rare articleType classes that occur only once
valid_article_types = df['articleType'].value_counts()
df = df[df['articleType'].isin(valid_article_types[valid_article_types > 1].index)]

# Stratified train/val split on cleaned data
train_df, val_df = train_test_split(
    df,
    test_size=0.15,
    random_state=42,
    stratify=df['articleType']
)

print(f"✅ Train shape: {train_df.shape}, Val shape: {val_df.shape}")
train_dataset = FashionDataset(train_df, img_dir, transform=get_train_augs())
val_dataset = FashionDataset(val_df, img_dir, transform=get_val_augs())

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

print(f"Training on {len(train_dataset)} samples, validating on {len(val_dataset)} samples.")

# --- 4. Initialize Model, Loss, and Optimizer ---
model = MultiTaskEfficientNet(
    num_classes['colors'], num_classes['types'], num_classes['seasons'], num_classes['genders']
).to(device)

# Using simple unweighted loss here, but you can add class weights if needed
criterion = {
    'color': nn.CrossEntropyLoss(),
    'product_type': nn.CrossEntropyLoss(),
    'season': nn.CrossEntropyLoss(),
    'gender': nn.CrossEntropyLoss()
}

optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.1, verbose=True)

# --- 5. Training & Validation Loop ---
best_val_loss = float('inf')
num_epochs = 10 

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    # Training phase
    model.train()
    total_train_loss = 0
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]")
    for images, targets in train_loop:
        images = images.to(device)
        targets = {k: v.to(device) for k, v in targets.items()}

        optimizer.zero_grad()
        outputs = model(images)
        loss = sum(criterion[task](outputs[task], targets[task]) for task in outputs)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        train_loop.set_postfix(loss=loss.item())

    # Validation phase
    model.eval()
    total_val_loss = 0
    val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]")
    with torch.no_grad():
        for images, targets in val_loop:
            images = images.to(device)
            targets = {k: v.to(device) for k, v in targets.items()}
            outputs = model(images)
            loss = sum(criterion[task](outputs[task], targets[task]) for task in outputs)
            total_val_loss += loss.item()

    # Calculate average losses
    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)
    
    print(f"\nEpoch {epoch+1}/{num_epochs} | Avg Train Loss: {avg_train_loss:.4f} | Avg Val Loss: {avg_val_loss:.4f}")
    
    scheduler.step(avg_val_loss)
    
    # Save the best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), model_save_path)
        print(f"Validation loss improved. Model saved to {model_save_path}")

print(" Training complete!")

Using device: cuda
✅ Label encoders saved to D:\CODING\Machine Learning\PROJECTS\fashion-product-classifier\backend\models\effi_net_backbone\label_encoders.json
✅ Train shape: (37385, 14), Val shape: (6598, 14)
Training on 37385 samples, validating on 6598 samples.


C:\Users\hamza\AppData\Local\Temp\ipykernel_35012\496865969.py:7: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=1, max_height=64, max_width=64, p=0.5),
C:\Users\hamza\anaconda3\envs\codemonk_env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10


Epoch 1/10 [Validation]: 100%|███████████████████████████████████████████████████████| 104/104 [09:58<00:00,  5.76s/it]



Epoch 1/10 | Avg Train Loss: 5.3480 | Avg Val Loss: 4.4483
Validation loss improved. Model saved to D:\CODING\Machine Learning\PROJECTS\fashion-product-classifier\backend\models\effi_net_backbone\model_efficientnet.pth
Epoch 2/10


...

Epoch 8/10 [Validation]: 100%|███████████████████████████████████████████████████████| 104/104 [08:29<00:00,  4.90s/it]



Epoch 8/10 | Avg Train Loss: 4.1820 | Avg Val Loss: 3.7370
Validation loss improved. Model saved to D:\CODING\Machine Learning\PROJECTS\fashion-product-classifier\backend\models\effi_net_backbone\model_efficientnet.pth
Epoch 9/10


Epoch 9/10 [Validation]: 100%|███████████████████████████████████████████████████████| 104/104 [07:54<00:00,  4.56s/it]



Epoch 9/10 | Avg Train Loss: 4.1619 | Avg Val Loss: 3.6951
Validation loss improved. Model saved to D:\CODING\Machine Learning\PROJECTS\fashion-product-classifier\backend\models\effi_net_backbone\model_efficientnet.pth
Epoch 10/10


Epoch 10/10 [Training]:  16%|███████▌                                      | 96/585 [07:44<44:59,  5.52s/it, loss=4.08]

In [ ]:
!